<a href="https://colab.research.google.com/github/daichisaito-cs/LMSYS/blob/main/LMSYS_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Setup

In [1]:
# driveのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Kaggle APIのインストール
!pip install kaggle

# マウントしたDriveの任意のフォルダに置いたkaggle.jsonファイルを
!cp "/content/drive/Shareddrives/株式会社inSane (1)/Workspace/kaggle/kaggle.json" /content/

# kaggle.jsonをColabのストレージにコピーする
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [6]:
!mkdir -p /content/input

!kaggle datasets download -d kishanvavdara/lmsys-model -p /content/input
!kaggle datasets download -d stochoshi/lsys-models-4 -p /content/input

!unzip /content/input/lmsys-model.zip -d /content/input/lmsys-model
!unzip /content/input/lsys-models-4.zip -d /content/input/lsys-models-4

Dataset URL: https://www.kaggle.com/datasets/kishanvavdara/lmsys-model
License(s): unknown
 91% 24.0M/26.4M [00:01<00:00, 33.1MB/s]
100% 26.4M/26.4M [00:01<00:00, 21.0MB/s]
Dataset URL: https://www.kaggle.com/datasets/stochoshi/lsys-models-4
License(s): CC0-1.0
100% 745M/748M [00:20<00:00, 40.5MB/s]
100% 748M/748M [00:20<00:00, 38.5MB/s]
Archive:  /content/input/lmsys-model.zip
  inflating: /content/input/lmsys-model/model  
  inflating: /content/input/lmsys-model/spmd_util.py  
  inflating: /content/input/lmsys-model/tokenizer/special_tokens_map.json  
  inflating: /content/input/lmsys-model/tokenizer/tokenizer.json  
  inflating: /content/input/lmsys-model/tokenizer/tokenizer_config.json  
Archive:  /content/input/lsys-models-4.zip
  inflating: /content/input/lsys-models-4/ccvec.pkl  
  inflating: /content/input/lsys-models-4/cvec.pkl  
  inflating: /content/input/lsys-models-4/hash  
  inflating: /content/input/lsys-models-4/lgb_models.pkl  
  inflating: /content/input/lsys-models-4

# Submission

In [ ]:
!kaggle competitions submit competitive-data-science-predict-future-sales -f "/content/drive/MyDrive/{hoge}" -m "{message}"

In [7]:
!ls

drive  input  sample_data
